In [1]:
# Load Data
#tbird_logs = sc.textFile("hdfs:///user/ytesfaye/tbird.log.out.logCluster.processed.gz")#.repartition(10)
tbird_logs = sc.textFile("hdfs:///user/ytesfaye/tbird.log.preProc.gz").repartition(10)

In [2]:
from collections import namedtuple
LogLine = namedtuple('LogLine', ['ts', 'msg'])
def get_tbird_data(line):
    ls = line.split(' ', 1)
    timestamp = float(ls[0])
    msg = ls[1]
    return LogLine(ts=timestamp, msg=msg)
tbird_loglines = tbird_logs.map(get_tbird_data).cache()

In [1]:
tbird_loglines.take(5)

In [4]:
# Get Words that occur more than support times 
from collections import defaultdict
support = 200
def parse_words(line):
    words = set(line.msg.split())
    return [(word, 1) for word in words]

frequentWordDict = tbird_loglines.flatMap(parse_words)\
                                 .reduceByKey(lambda x,y: x+y)\
                                 .filter(lambda (key,count): count > support)\
                                 .collectAsMap()
frequentWords = sc.broadcast(set(frequentWordDict.keys()))

In [5]:
def extract_patterns(line):
    skip = 0
    freqWordPattern = []
    pattern = []
    for word in line.msg.split():
        if word in frequentWords.value:
            if skip != 0:
                pattern.append(skip)
                skip = 0
            freqWordPattern.append(word)
            pattern.append(word)
        else:
            skip += 1    
    return (tuple(freqWordPattern), pattern)

In [6]:
def parse_list(pattern):
    freqWordPattern, patterns = pattern
    patterns = set([tuple(pattern) for pattern in patterns])
    aggregate_pattern = [set()]
    for word in freqWordPattern:
        aggregate_pattern.append(word)
        aggregate_pattern.append(set())
    
    for pattern in patterns:
        output_loc = 0
        prev_val = 0
        for word in pattern:
            if isinstance(word, int):
                aggregate_pattern[output_loc].add(word)
                output_loc += 1
                prev_val = 1
            else:
                # TODO: Add check here that it matches what it should match
                #if word != aggregate_pattern[output_loc+1]:
                #    print 'ERROR', word, aggregate_pattern[output_loc]
                if prev_val == 0:
                    aggregate_pattern[output_loc].add(0)
                    output_loc += 2
                else:
                    output_loc += 1
                prev_val = 0
    
    final_pattern = []
    for word in aggregate_pattern:
        if isinstance(word, set):
            if len(word) >= 2:
                final_pattern.append('(:? S+){%d,%d}'%(min(word), max(word)))
            elif len(word) == 1 and 0 not in word: # Always skip the same number of values
                final_pattern.append('(:? S+){%d,%d}'%(min(word), max(word)))
        else:
            final_pattern.append(word)
    return final_pattern

In [7]:
t = tbird_loglines.map(extract_patterns)\
                  .groupByKey()\
                  .filter(lambda (freqWordPattern, pattern): len(pattern) > support)\
                  .map(parse_list)\
                  .collect()

In [10]:
for l in sorted(t):
    print ' '.join(l)

# INT # crond(pam_unix)[ INT ]: session closed for user USER
# INT # crond(pam_unix)[ INT ]: session opened for user USER by ( KEYVALUE INT )
# INT # crond[ INT ]: ( USER ) CMD (run-parts FILEPATH
# INT # exiting on signal INT
# INT # gmond: gmond shutdown succeeded
# INT # gmond: gmond startup succeeded
# INT # kernel: audit( FILEANDLINE ): avc: denied { append } for KEYVALUE INT KEYVALUE FILEPATH KEYVALUE KEYVALUE INT KEYVALUE :system_r:ntpd_t KEYVALUE :object_r:ramfs_t KEYVALUE
# INT # kernel: audit( FILEANDLINE ): avc: denied { getattr } for KEYVALUE INT KEYVALUE FILEPATH KEYVALUE INT KEYVALUE INT KEYVALUE :system_r:ntpd_t KEYVALUE :object_r:file_t KEYVALUE
# INT # kernel: audit( FILEANDLINE ): avc: denied { read write } for KEYVALUE INT KEYVALUE KEYVALUE :[ INT ] KEYVALUE KEYVALUE INT KEYVALUE :system_r:ntpd_t KEYVALUE :system_r:unconfined_t KEYVALUE
# INT # kernel: audit( FILEANDLINE ): avc: denied { read write } for KEYVALUE INT KEYVALUE KEYVALUE KEYVALUE INT KEYVALUE INT KEYVAL

In [11]:
len(t)

662

In [69]:
import re
print re.escape(' '.join(parse_list(t[0])))

\(\:\?\ S\+\)\{0\,1\}\ checking\ root\ filesystem\ succeeded


In [71]:
for i in t:
    print ' '.join(parse_list(i))[:60]

(:? S+){0,1} checking root filesystem succeeded
endmail INT jac INT INT to root ctladdr root FILEPATH delay 
l
AFILE INT FILEANDLINE program port state node INT a INT (:? 
pam_unix INT session closed for user mwsukal
l uhci_hcd INT INT AFILE irq INT io base INT acc INT
d smartd startup succeeded
l dcdbas bios update phys addr INT e INT
emon haldaemon startup succeeded
l ide assuming INT mhz system bus speed for pio modes; overr
l divert allocating divert_blk for eth INT
d INT removing klogin
(:? S+){0,1} INT got trap from peer on fd INT
l real time clock driver AFILE
INT listening on interface eth INT IPADDR # INT
INT FILEPATH unable to write meta data for metric cpu_nice t
l using acpi madt for smp configuration LEVEL rmation
AFILE INT LEVEL ib clisessionthread pid INT
INT FILEPATH LEVEL ing unable to look up FILEPATH no such fi
l uniform multi-platform e-ide driver revision AFILE
l acpi power button ff pwrf
AFILE INT FILEANDLINE failed to set lft to switch INT a INT
l acpi ioapic id 

In [72]:
r = list(t[1][1])

In [8]:
for i in range(len(t)):
    if t[i][0][0] == 'USER':
        print t[i][0]

In [84]:
print t[3][0][0]

eadmin


In [14]:
list(t[0][1])

[[u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'filesystem', u'succeeded'],
 [u'checking', u'root', u'fi